### Imports

In [55]:
# Replaced with StyleFormer framework
!pip install styleformer transformers

### Import Necessary Libraries

In [57]:
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
from sklearn.metrics import accuracy_score, classification_report
from torch.nn.functional import softmax
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

In [58]:
# Replaced with StyleFormer framework
# Check if GPU is available and set device appropriately
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Styleformer for formal to informal style transfer
style_transformer = Styleformer(style=0)  # 0 for formal to informal

Casual to Formal model loaded...


In [59]:
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file.readlines()]

### Load the text files

In [61]:
train_texts = load_data('/Users/xuqianlong/Downloads/train_text.txt')
train_labels = np.array(load_data('/Users/xuqianlong/Downloads/train_labels.txt'), dtype=int)
val_texts = load_data('/Users/xuqianlong/Downloads/val_text.txt')
val_labels = np.array(load_data('/Users/xuqianlong/Downloads/val_labels.txt'), dtype=int)
test_texts = load_data('/Users/xuqianlong/Downloads/test_text.txt')
test_labels = np.array(load_data('/Users/xuqianlong/Downloads/test_labels.txt'), dtype=int)

In [62]:
#Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [63]:
# Encode data
def encode_texts(texts, labels):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens = True,
            max_length = 64,
            pad_to_max_length = True,
            return_attention_mask = True,
            return_tensors = 'pt',
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    return input_ids, attention_masks, labels

In [64]:
# Prepare data set
train_inputs, train_masks, train_labels = encode_texts(train_texts, train_labels)
val_inputs, val_masks, val_labels = encode_texts(val_texts, val_labels)
test_inputs, test_masks, test_labels = encode_texts(test_texts, test_labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [65]:
# Create TensorDatasets
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)

In [66]:
# Model initialization
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
#Run the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [68]:
#Train model
epochs = 4
batch_size = 32
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataset) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
def train_model(model, train_dataset, val_dataset, epochs=4, batch_size=32):
    train_dataloader = DataLoader(
        train_dataset,
        sampler=RandomSampler(train_dataset),
        batch_size=batch_size
    )
    validation_dataloader = DataLoader(
        val_dataset,
        sampler=SequentialSampler(val_dataset),
        batch_size=batch_size
    )

    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    # Put the model on the GPU
    model.to(device)
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        print('-' * 10)

        # training phase
        model.train()
        total_train_loss = 0

        for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            model.zero_grad()        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels)
            loss = outputs.loss
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"  Average training loss: {avg_train_loss:.2f}")

        # Verification phase
        model.eval()
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        for batch in tqdm(validation_dataloader, desc="Validating"):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            with torch.no_grad():        
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask,
                                labels=b_labels)
                loss = outputs.loss
                logits = outputs.logits
                
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            total_eval_accuracy += flat_accuracy(logits, label_ids)

        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print(f"  Validation Accuracy: {avg_val_accuracy:.2f}")
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        print(f"  Validation Loss: {avg_val_loss:.2f}")

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Run training
train_model(model, train_dataset, val_dataset, epochs=4)

Epoch 1/4
----------


Training: 100%|██████████| 1426/1426 [47:40<00:00,  2.01s/it]


  Average training loss: 0.66


Validating: 100%|██████████| 63/63 [00:37<00:00,  1.69it/s]


  Validation Accuracy: 0.74
  Validation Loss: 0.61
Epoch 2/4
----------


...

  Validation Accuracy: 0.74
  Validation Loss: 0.75
Epoch 4/4
----------


Training: 100%|██████████| 1426/1426 [45:59<00:00,  1.93s/it]


  Average training loss: 0.24


Validating: 100%|██████████| 63/63 [00:31<00:00,  1.97it/s]

  Validation Accuracy: 0.74
  Validation Loss: 0.86


In [ ]:
# Create the DataLoader for testing
test_dataloader = DataLoader(
    test_dataset,
    batch_size=16,  # Adjust this batch size based on your system's capabilities
    shuffle=False  # No need to shuffle the test data
)
# Model evaluation
def evaluate_model(model, test_dataset):
    # Test code
    model.eval()
    predictions , true_labels = [], []
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.append(logits)
        true_labels.append(label_ids)

    # Calculate accuracy and other indicators
    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    pred_labels = np.argmax(predictions, axis=1)
    print(classification_report(true_labels, pred_labels))
    return accuracy_score(true_labels, pred_labels)
# Call the evaluation function
test_accuracy = evaluate_model(model, test_dataset)
print(f"Test Accuracy: {test_accuracy}")